In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor()])
    

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [4]:
orig_data = datasets.MNIST('./data/', train=True, download=True, transform=transform)

In [5]:
dloader = DataLoader(dataset=orig_data, batch_size=10)

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.l1 = nn.Linear(in_features=128, out_features=512)
        self.l2 = nn.Linear(in_features=512, out_features=784)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # Expecting x to be of 128 * 1
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.sigmoid(x)
        return x

In [30]:
def getZ(device):
    var = torch.rand((10, 128))
    var = var.view(10, 1, 128)
    var = var.to(device)
    return var

In [8]:
var = getZ(device)

In [9]:
var.size()

torch.Size([10, 1, 128])

In [10]:
gen = Generator()

In [11]:
x = gen(var)

In [12]:
x.size()

torch.Size([10, 1, 784])

In [13]:
class Disc(nn.Module):
    def __init__(self):
        super(Disc, self).__init__()
        self.l1 = nn.Linear(in_features=784, out_features=256)
        self.l2 = nn.Linear(in_features=256, out_features=1)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.sigmoid(x)
        return x

In [14]:
disc = Disc()

In [15]:
op = disc(x)

In [16]:
op.size()

torch.Size([10, 1, 1])

In [17]:
discLoss = nn.BCELoss()

In [18]:
genLoss = nn.BCELoss()

In [19]:
gen = gen.to(device)

In [20]:
disc = disc.to(device)

In [21]:
dOptim = torch.optim.Adam(disc.parameters(), lr=0.0002)
gOptim = torch.optim.Adam(gen.parameters(), lr=0.0002)

In [22]:
def getLabels(bsize, device, orig=True):
    op = None
    if orig:
        op = torch.ones(bsize, 1)
    else:
        op = torch.zeros(bsize, 1)
    op = op.to(device)
    return op

In [23]:
epochs = 108

In [38]:
posLabel = getLabels(10, device)
fakeLabel = getLabels(10, device, False)

In [42]:
for e in range(epochs):
    for batch, _ in dloader:
        batch = batch.to(device)
        batch = batch.view(10, 1, -1)
        
        dOptim.zero_grad()
        gOptim.zero_grad()
        
        z = getZ(device)
        z = gen(z)
        
        fakeD = disc(z)
        origD = disc(batch)
        
        dFakeLoss = discLoss(fakeD, fakeLabel)
        dOrigLoss = discLoss(origD, posLabel)
        
        dloss = dFakeLoss + dOrigLoss
        gLoss = genLoss(fakeD, posLabel)
        
        assert False

AssertionError: 